In [1]:
library(readxl)
library(tidyverse)
library(igraph)

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘igraph’


The following objects are masked from ‘package:lubridate’:

    %--%, union


The following objects are masked from ‘package:dplyr’:

    as_data_frame, groups, union


The following objects are masked from ‘package:purrr’:

    compose, simplify


The following object is masked from ‘package:tidyr’:

    crossing


The following object is m

# Data exploration

## Expression_data.xlsx
It contains quantification of individual proteins (columns) in the various samples (rows). Identifiers for protein are from UniProt. The last number in the sample name indicates replicates in the sample group (e.g. S0R24h_4 is the fourth replicate of the S0R24h group).

In [2]:
expression_data <- read_xlsx("data/expression_data.xlsx")
names(expression_data)[names(expression_data) == "...1"] <- "sample_id"
expression_data <- expression_data %>%
    add_column(
        group = substr(expression_data$sample_id, 1, str_locate(expression_data$sample_id, "_") - 1), 
        repl = substr(expression_data$sample_id, str_locate(expression_data$sample_id, "_") + 1, nchar(expression_data$sample_id)),  
        .after = 1)
expression_data

New names:
• `` -> `...1`


sample_id,group,repl,A0A067XG53,A0A075B5L5,A0A075B5P4,A0A075B5P5,A0A075B5P6,A0A087WNV1,A0A087WP47,⋯,Z4YJE9,Z4YJT3,Z4YJZ7,Z4YKC4,Z4YKM2,Z4YKT6,Z4YL78,Z4YLT8,Z4YN00,Z4YNA3
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
S0R24h_1,S0R24h,1,43488.0,0.0,0.0,20276.0,108610.0,458490,209890.00,⋯,177570.0,2319800,94586.0,97746.0,59840,100420.0,1116400,37164.0,211870,0.0
S0R24h_2,S0R24h,2,55503.0,0.0,0.0,23570.0,134020.0,405240,160100.00,⋯,184240.0,2112400,133110.0,111870.0,70452,28561.0,1027600,3060.2,206690,0.0
S0R24h_3,S0R24h,3,52803.0,0.0,0.0,21050.0,134920.0,462110,106090.00,⋯,140150.0,554320,50077.0,60430.0,12682,7099.9,626490,2284.8,251290,0.0
S0R24h_4,S0R24h,4,42735.0,0.0,7592.3,12950.0,106180.0,323430,222370.00,⋯,155370.0,2397100,43383.0,95524.0,64375,48862.0,581480,20364.0,227450,508970.0
S0R24h_5,S0R24h,5,47269.0,0.0,0.0,22438.0,112100.0,393260,140350.00,⋯,157810.0,2231300,95853.0,78070.0,58544,25261.0,1017200,13811.0,193340,0.0
S0R24h_6,S0R24h,6,41319.0,0.0,0.0,27004.0,182070.0,283180,197910.00,⋯,175480.0,1965100,49407.0,77490.0,36523,92820.0,908840,18727.0,174270,0.0
S0R48h_1,S0R48h,1,13286.0,0.0,0.0,26524.0,48645.0,353740,73440.00,⋯,119980.0,1611700,20867.0,24792.0,31985,53277.0,561020,77070.0,144590,0.0
S0R48h_2,S0R48h,2,22146.0,0.0,0.0,19478.0,61485.0,221960,115830.00,⋯,140460.0,1807500,80323.0,37901.0,48602,99953.0,847190,78374.0,136730,640120.0
S0R48h_3,S0R48h,3,16531.0,0.0,0.0,27020.0,55285.0,272090,92319.00,⋯,99222.0,1432500,0.0,25349.0,21962,56371.0,456720,0.0,121670,62124.0


In [3]:

expression_data_proteins <- colnames(expression_data)[4:ncol(expression_data)]
expression_data_proteins


[1] "A0A067XG53" "A0A075B5L5" "A0A075B5P4" "A0A075B5P5" "A0A075B5P6"
   [6] "A0A087WNV1" "A0A087WP47" "A0A087WPF0" "A0A087WPP8" "A0A087WPT7"
  [11] "A0A087WPZ6" "A0A087WQM0" "A0A087WQN7" "A0A087WQS2" "A0A087WR20"
  [16] "A0A087WR42" "A0A087WRH2" "A0A087WRY3" "A0A087WS03" "A0A0A0MQ68"
  [21] "A0A0A0MQ79" "A0A0A0MQ90" "A0A0A0MQ98" "A0A0A0MQ99" "A0A0A0MQA5"
  [26] "A0A0A0MQB0" "A0A0A6YVS2" "A0A0A6YWB0" "A0A0A6YWP6" "A0A0A6YWS6"
  [31] "A0A0A6YWX1" "A0A0A6YX02" "A0A0A6YX73" "A0A0A6YXC8" "A0A0A6YXL6"
  [36] "A0A0A6YXW7" "A0A0A6YY47" "A0A0B4J1G1" "A0A0B4J1G5" "A0A0B4J1L4"
  [41] "A0A0B4J1M6" "A0A0D2X7Z2" "A0A0F6AIX5" "A0A0G2JDI9" "A0A0G2JDW1"
  [46] "A0A0G2JDW7" "A0A0G2JE26" "A0A0G2JE99" "A0A0G2JEC4" "A0A0G2JEH5"
  [51] "A0A0G2JEI3" "A0A0G2JEK2" "A0A0G2JES4" "A0A0G2JF16" "A0A0G2JFC9"
  [56] "A0A0G2JFF9" "A0A0G2JFH0" "A0A0G2JFJ6" "A0A0G2JFY0" "A0A0G2JFZ4"
  [61] "A0A0G2JG52" "A0A0G2JG82" "A0A0G2JGJ1" "A0A0G2JGL8" "A0A0G2JGQ6"
  [66] "A0A0G2JH06" "A0A0G2JH18" "A0A0J9YU62" "A0A0J9YUE9" "A0A0J9YUR2"
  [71] "A0A0J9YUW6" "A0A0J9YUX0" "A0A0J9YV01" "A0A0J9YV20" "A0A0N4SUV6"
  [76] "A0A0N4SV29" "A0A0N4SVB8" "A0A0N4SVC2" "A0A0N4SVL9" "A0A0N4SVT1"
  [81] "A0A0N4SVT7" "A0A0N4SW89" "A0A0N5E9G7" "A0A0R4IZW9" "A0A0R4IZX1"
  [86] "A0A0R4IZY0" "A0A0R4IZY6" "A0A0R4IZY9" "A0A0R4J007" "A0A0R4J018"
  [91] "A0A0R4J023" "A0A0R4J047" "A0A0R4J052" "A0A0R4J058" "A0A0R4J060"
  [96] "A0A0R4J069" "A0A0R4J085" "A0A0R4J091" "A0A0R4J094" "A0A0R4J097"
 [101] "A0A0R4J098" "A0A0R4J099" "A0A0R4J0B2" "A0A0R4J0B4" "A0A0R4J0C8"
 [106] "A0A0R4J0D3" "A0A0R4J0F6" "A0A0R4J0G0" "A0A0R4J0G4" "A0A0R4J0G9"
 [111] "A0A0R4J0I0" "A0A0R4J0J5" "A0A0R4J0K1" "A0A0R4J0M1" "A0A0R4J0P1"
 [116] "A0A0R4J0P5" "A0A0R4J0P6" "A0A0R4J0R3" "A0A0R4J0S1" "A0A0R4J0S4"
 [121] "A0A0R4J0T4" "A0A0R4J0U7" "A0A0R4J0V5" "A0A0R4J0W6" "A0A0R4J0X5"
 [126] "A0A0R4J0Z1" "A0A0R4J124" "A0A0R4J131" "A0A0R4J138" "A0A0R4J139"
 [131] "A0A0R4J169" "A0A0R4J170" "A0A0R4J175" "A0A0R4J187" "A0A0R4J195"
 [136] "A0A0R4J1B9" "A0A0R4J1D0" "A0A0R4J1E3" "A0A0R4J1H0" "A0A0R4J1N8"
 [141] "A0A0R4J1N9" "A0A0R4J1R7" "A0A0R4J1Y3" "A0A0R4J205" "A0A0R4J254"
 [146] "A0A0R4J257" "A0A0R4J260" "A0A0R4J2D3" "A0A0U1RNK7" "A0A0U1RNL1"
 [151] "A0A0U1RNL7" "A0A0U1RNS6" "A0A0U1RNV6" "A0A0U1RP59" "A0A0U1RPD7"
 [156] "A0A0U1RPG9" "A0A0U1RPM2" "A0A0U1RPS0" "A0A0U1RQ36" "A0A0U1RQ85"
 [161] "A0A0U1RQA4" "A0A0U1RQA8" "A0A140LHL5" "A0A140LHU9" "A0A140LHY2"
 [166] "A0A140LIM2" "A0A140LIQ1" "A0A140LIT9" "A0A140LJK0" "A0A140T8J4"
 [171] "A0A140T8R8" "A0A140T8V9" "A0A1B0GQZ1" "A0A1B0GR08" "A0A1B0GR11"
 [176] "A0A1B0GR19" "A0A1B0GR60" "A0A1B0GR65" "A0A1B0GR85" "A0A1B0GRC0"
 [181] "A0A1B0GRH2" "A0A1B0GRV0" "A0A1B0GRV3" "A0A1B0GS41" "A0A1B0GS63"
 [186] "A0A1B0GSE5" "A0A1B0GSH8" "A0A1B0GSV4" "A0A1B0GSX7" "A0A1B0GT07"
 [191] "A0A1B0GT75" "A0A1B0GT78" "A0A1B0GT81" "A0A1B0GT92" "A0A1C7CYV0"
 [196] "A0A1D5RLL3" "A0A1D5RLR3" "A0A1D5RLS1" "A0A1D5RLX7" "A0A1D5RM61"
 [201] "A0A1D5RM90" "A0A1D5RMA9" "A0A1D5RMB5" "A0A1D5RMC8" "A0A1D5RMM1"
 [206] "A0A1I7Q4G8" "A0A1L1SRC1" "A0A1L1SRE8" "A0A1L1SRG5" "A0A1L1SRW2"
 [211] "A0A1L1SRX2" "A0A1L1SS10" "A0A1L1SSA8" "A0A1L1SSY9" "A0A1L1STE4"
 [216] "A0A1L1STE6" "A0A1L1STF4" "A0A1L1SUX8" "A0A1L1SVD8" "A0A1L1SVG0"
 [221] "A0A1N9M8A6" "A0A1Q1PRS4" "A0A1S6GWJ4" "A0A1U6URG8" "A0A1W2P6X3"
 [226] "A0A1W2P737" "A0A1W2P7K6" "A0A1W2P7Z1" "A0A1W2P845" "A0A1Y7VJ48"
 [231] "A0A1Y7VKC0" "A0A1Y7VKT9" "A0A1Y7VMS4" "A0A1Y7VNT1" "A0A1Y7VP73"
 [236] "A0A286YCI8" "A0A286YCQ5" "A0A286YCX6" "A0A286YD58" "A0A286YD76"
 [241] "A0A286YD87" "A0A286YDB3" "A0A286YDB7" "A0A286YDB8" "A0A286YDC0"
 [246] "A0A286YDG5" "A0A286YDN1" "A0A286YDP0" "A0A286YDT0" "A0A286YDZ5"
 [251] "A0A2C9F2A2" "A0A2I3BPM7" "A0A2I3BPN6" "A0A2I3BPT1" "A0A2I3BPX3"
 [256] "A0A2I3BPX9" "A0A2I3BQ07" "A0A2I3BQE0" "A0A2I3BQS6" "A0A2I3BQU5"
 [261] "A0A2I3BRD1" "A0A2I3BRR5" "A0A2I3BRT5" "A0A2I3BRW0" "A0A2K6EDJ7"
 [266] "A0A2R8VHH1" "A0A2R8VHL2" "A0A2R8VHQ0" "A0A2R8VHU8" "A0A2R8VHX0"
 [271] "A0A2R8VI30" "A0A2R8VI72" "A0A2R8VJW3" "A0A2R8VKP3" "A0A2R8W6R3"
 [276] "A0A338P684" "A0A338P6K3" "A0A338P6N

## Comparisons.txt
Shows what sample groups we analyzed (for statistical tests) - it's mainly for context, because as we talked about, we would like to come up with an approach that might consider all sample groups when training, rather than being based on pairs of sample groups as in the current method (statistical testing for differential expression, and pathway enrichment).

In [4]:
comparisons <- read_delim("data/Comparisons.txt", col_names = c("group1", "group2"), delim = " vs ")
comparisons

Rows: 5 Columns: 2
── Column specification ────────────────────────────────────────────────────────
Delimiter: " vs "
chr (2): group1, group2

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


group1,group2
<chr>,<chr>
T66,T67
S0R48h,S400R48h
S4R48h,S400R48h
S0R24h,S400R24h
S4R24h,S400R24h


## MMU_Uniprot2Reactome.txt

tab separated; in order, the columns contain the uniprot ID of proteins, the Reactome Pathway ID they belong to, a link, and additional information about the pathway/reaction (text description, a code, and the organism - only mouse in this case)

In [5]:
uniprot_to_reactome <- read_tsv(
    "data/MMU_Uniprot2Reactome.txt",
    col_names = c("uniprot_id", "reactome_pathway_id", "reactome_pathway_url", "description", "code", "organism"),
    skip = 1
    )

uniprot_to_reactome

Rows: 79497 Columns: 6
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (6): uniprot_id, reactome_pathway_id, reactome_pathway_url, description,...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


uniprot_id,reactome_pathway_id,reactome_pathway_url,description,code,organism
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
A0A075B5J3,R-MMU-198955,https://reactome.org/PathwayBrowser/#/R-MMU-198955,TCR complex interacts with peptide antigen-presenting MHC Class I,IEA,Mus musculus
A0A075B5J3,R-MMU-202165,https://reactome.org/PathwayBrowser/#/R-MMU-202165,Phosphorylation of ITAM motifs in CD3 complexes,IEA,Mus musculus
A0A075B5J3,R-MMU-202168,https://reactome.org/PathwayBrowser/#/R-MMU-202168,Phosphorylation of ZAP-70 by Lck,IEA,Mus musculus
A0A075B5J3,R-MMU-202174,https://reactome.org/PathwayBrowser/#/R-MMU-202174,Activation of ZAP-70,IEA,Mus musculus
A0A075B5J3,R-MMU-202214,https://reactome.org/PathwayBrowser/#/R-MMU-202214,Dephosphorylation of Lck-pY505 by CD45,IEA,Mus musculus
A0A075B5J3,R-MMU-202216,https://reactome.org/PathwayBrowser/#/R-MMU-202216,Phosphorylation of SLP-76,IEA,Mus musculus
A0A075B5J3,R-MMU-202233,https://reactome.org/PathwayBrowser/#/R-MMU-202233,Inactivation of Lck by Csk,IEA,Mus musculus
A0A075B5J3,R-MMU-202245,https://reactome.org/PathwayBrowser/#/R-MMU-202245,Phosphorylation of TBSMs in LAT,IEA,Mus musculus
A0A075B5J3,R-MMU-202248,https://reactome.org/PathwayBrowser/#/R-MMU-202248,Phosphorylation of PLC-gamma1,IEA,Mus musculus


In [6]:
uniprot_to_reactome %>%
    select(uniprot_id) %>%
    distinct()

uniprot_id
<chr>
A0A075B5J3
A0A075B5J4
A0A075B5J7
A0A075B5J9
A0A075B5K0
A0A075B5K3
A0A075B5K7
A0A075B5K8
A0A075B5K9


In [7]:
uniprot_to_reactome %>%
    select(reactome_pathway_id) %>%
    distinct()

reactome_pathway_id
<chr>
R-MMU-198955
R-MMU-202165
R-MMU-202168
R-MMU-202174
R-MMU-202214
R-MMU-202216
R-MMU-202233
R-MMU-202245
R-MMU-202248


In [8]:
uniprot_to_reactome %>%
    select(uniprot_id,reactome_pathway_id) %>%
    distinct()

uniprot_id,reactome_pathway_id
<chr>,<chr>
A0A075B5J3,R-MMU-198955
A0A075B5J3,R-MMU-202165
A0A075B5J3,R-MMU-202168
A0A075B5J3,R-MMU-202174
A0A075B5J3,R-MMU-202214
A0A075B5J3,R-MMU-202216
A0A075B5J3,R-MMU-202233
A0A075B5J3,R-MMU-202245
A0A075B5J3,R-MMU-202248


In [16]:
# get column names from expression data
cols <- colnames(expression_data)[4:ncol(expression_data)]
cols

[1] "A0A067XG53" "A0A075B5L5" "A0A075B5P4" "A0A075B5P5" "A0A075B5P6"
   [6] "A0A087WNV1" "A0A087WP47" "A0A087WPF0" "A0A087WPP8" "A0A087WPT7"
  [11] "A0A087WPZ6" "A0A087WQM0" "A0A087WQN7" "A0A087WQS2" "A0A087WR20"
  [16] "A0A087WR42" "A0A087WRH2" "A0A087WRY3" "A0A087WS03" "A0A0A0MQ68"
  [21] "A0A0A0MQ79" "A0A0A0MQ90" "A0A0A0MQ98" "A0A0A0MQ99" "A0A0A0MQA5"
  [26] "A0A0A0MQB0" "A0A0A6YVS2" "A0A0A6YWB0" "A0A0A6YWP6" "A0A0A6YWS6"
  [31] "A0A0A6YWX1" "A0A0A6YX02" "A0A0A6YX73" "A0A0A6YXC8" "A0A0A6YXL6"
  [36] "A0A0A6YXW7" "A0A0A6YY47" "A0A0B4J1G1" "A0A0B4J1G5" "A0A0B4J1L4"
  [41] "A0A0B4J1M6" "A0A0D2X7Z2" "A0A0F6AIX5" "A0A0G2JDI9" "A0A0G2JDW1"
  [46] "A0A0G2JDW7" "A0A0G2JE26" "A0A0G2JE99" "A0A0G2JEC4" "A0A0G2JEH5"
  [51] "A0A0G2JEI3" "A0A0G2JEK2" "A0A0G2JES4" "A0A0G2JF16" "A0A0G2JFC9"
  [56] "A0A0G2JFF9" "A0A0G2JFH0" "A0A0G2JFJ6" "A0A0G2JFY0" "A0A0G2JFZ4"
  [61] "A0A0G2JG52" "A0A0G2JG82" "A0A0G2JGJ1" "A0A0G2JGL8" "A0A0G2JGQ6"
  [66] "A0A0G2JH06" "A0A0G2JH18" "A0A0J9YU62" "A0A0J9YUE9" "A0A0J9YUR2"
  [71] "A0A0J9YUW6" "A0A0J9YUX0" "A0A0J9YV01" "A0A0J9YV20" "A0A0N4SUV6"
  [76] "A0A0N4SV29" "A0A0N4SVB8" "A0A0N4SVC2" "A0A0N4SVL9" "A0A0N4SVT1"
  [81] "A0A0N4SVT7" "A0A0N4SW89" "A0A0N5E9G7" "A0A0R4IZW9" "A0A0R4IZX1"
  [86] "A0A0R4IZY0" "A0A0R4IZY6" "A0A0R4IZY9" "A0A0R4J007" "A0A0R4J018"
  [91] "A0A0R4J023" "A0A0R4J047" "A0A0R4J052" "A0A0R4J058" "A0A0R4J060"
  [96] "A0A0R4J069" "A0A0R4J085" "A0A0R4J091" "A0A0R4J094" "A0A0R4J097"
 [101] "A0A0R4J098" "A0A0R4J099" "A0A0R4J0B2" "A0A0R4J0B4" "A0A0R4J0C8"
 [106] "A0A0R4J0D3" "A0A0R4J0F6" "A0A0R4J0G0" "A0A0R4J0G4" "A0A0R4J0G9"
 [111] "A0A0R4J0I0" "A0A0R4J0J5" "A0A0R4J0K1" "A0A0R4J0M1" "A0A0R4J0P1"
 [116] "A0A0R4J0P5" "A0A0R4J0P6" "A0A0R4J0R3" "A0A0R4J0S1" "A0A0R4J0S4"
 [121] "A0A0R4J0T4" "A0A0R4J0U7" "A0A0R4J0V5" "A0A0R4J0W6" "A0A0R4J0X5"
 [126] "A0A0R4J0Z1" "A0A0R4J124" "A0A0R4J131" "A0A0R4J138" "A0A0R4J139"
 [131] "A0A0R4J169" "A0A0R4J170" "A0A0R4J175" "A0A0R4J187" "A0A0R4J195"
 [136] "A0A0R4J1B9" "A0A0R4J1D0" "A0A0R4J1E3" "A0A0R4J1H0" "A0A0R4J1N8"
 [141] "A0A0R4J1N9" "A0A0R4J1R7" "A0A0R4J1Y3" "A0A0R4J205" "A0A0R4J254"
 [146] "A0A0R4J257" "A0A0R4J260" "A0A0R4J2D3" "A0A0U1RNK7" "A0A0U1RNL1"
 [151] "A0A0U1RNL7" "A0A0U1RNS6" "A0A0U1RNV6" "A0A0U1RP59" "A0A0U1RPD7"
 [156] "A0A0U1RPG9" "A0A0U1RPM2" "A0A0U1RPS0" "A0A0U1RQ36" "A0A0U1RQ85"
 [161] "A0A0U1RQA4" "A0A0U1RQA8" "A0A140LHL5" "A0A140LHU9" "A0A140LHY2"
 [166] "A0A140LIM2" "A0A140LIQ1" "A0A140LIT9" "A0A140LJK0" "A0A140T8J4"
 [171] "A0A140T8R8" "A0A140T8V9" "A0A1B0GQZ1" "A0A1B0GR08" "A0A1B0GR11"
 [176] "A0A1B0GR19" "A0A1B0GR60" "A0A1B0GR65" "A0A1B0GR85" "A0A1B0GRC0"
 [181] "A0A1B0GRH2" "A0A1B0GRV0" "A0A1B0GRV3" "A0A1B0GS41" "A0A1B0GS63"
 [186] "A0A1B0GSE5" "A0A1B0GSH8" "A0A1B0GSV4" "A0A1B0GSX7" "A0A1B0GT07"
 [191] "A0A1B0GT75" "A0A1B0GT78" "A0A1B0GT81" "A0A1B0GT92" "A0A1C7CYV0"
 [196] "A0A1D5RLL3" "A0A1D5RLR3" "A0A1D5RLS1" "A0A1D5RLX7" "A0A1D5RM61"
 [201] "A0A1D5RM90" "A0A1D5RMA9" "A0A1D5RMB5" "A0A1D5RMC8" "A0A1D5RMM1"
 [206] "A0A1I7Q4G8" "A0A1L1SRC1" "A0A1L1SRE8" "A0A1L1SRG5" "A0A1L1SRW2"
 [211] "A0A1L1SRX2" "A0A1L1SS10" "A0A1L1SSA8" "A0A1L1SSY9" "A0A1L1STE4"
 [216] "A0A1L1STE6" "A0A1L1STF4" "A0A1L1SUX8" "A0A1L1SVD8" "A0A1L1SVG0"
 [221] "A0A1N9M8A6" "A0A1Q1PRS4" "A0A1S6GWJ4" "A0A1U6URG8" "A0A1W2P6X3"
 [226] "A0A1W2P737" "A0A1W2P7K6" "A0A1W2P7Z1" "A0A1W2P845" "A0A1Y7VJ48"
 [231] "A0A1Y7VKC0" "A0A1Y7VKT9" "A0A1Y7VMS4" "A0A1Y7VNT1" "A0A1Y7VP73"
 [236] "A0A286YCI8" "A0A286YCQ5" "A0A286YCX6" "A0A286YD58" "A0A286YD76"
 [241] "A0A286YD87" "A0A286YDB3" "A0A286YDB7" "A0A286YDB8" "A0A286YDC0"
 [246] "A0A286YDG5" "A0A286YDN1" "A0A286YDP0" "A0A286YDT0" "A0A286YDZ5"
 [251] "A0A2C9F2A2" "A0A2I3BPM7" "A0A2I3BPN6" "A0A2I3BPT1" "A0A2I3BPX3"
 [256] "A0A2I3BPX9" "A0A2I3BQ07" "A0A2I3BQE0" "A0A2I3BQS6" "A0A2I3BQU5"
 [261] "A0A2I3BRD1" "A0A2I3BRR5" "A0A2I3BRT5" "A0A2I3BRW0" "A0A2K6EDJ7"
 [266] "A0A2R8VHH1" "A0A2R8VHL2" "A0A2R8VHQ0" "A0A2R8VHU8" "A0A2R8VHX0"
 [271] "A0A2R8VI30" "A0A2R8VI72" "A0A2R8VJW3" "A0A2R8VKP3" "A0A2R8W6R3"
 [276] "A0A338P684" "A0A338P6K3" "A0A338P6N

In [ ]:
# check what is in uniprot_to_reactome but not in expression_data_proteins
uniprot_to_reactome %>%
    select(uniprot_id) %>%
    distinct() %>%
    filter(!uniprot_id %in% cols)

## MMU_ReactomePathwaysRelation.txt
tab separated, links pairs of mouse pathways/reactions from reactome (from -> to)

In [9]:
reactome_pathways <- read_tsv(
    "data/MMU_ReactomePathwaysRelation.txt", 
    col_names = c("reactome_pathway_id", "reactome_reaction_id"))

reactome_pathways


Rows: 1740 Columns: 2
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (2): reactome_pathway_id, reactome_reaction_id

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


reactome_pathway_id,reactome_reaction_id
<chr>,<chr>
R-MMU-109581,R-MMU-109606
R-MMU-109581,R-MMU-169911
R-MMU-109581,R-MMU-5357769
R-MMU-109581,R-MMU-75153
R-MMU-109582,R-MMU-140877
R-MMU-109582,R-MMU-202733
R-MMU-109582,R-MMU-418346
R-MMU-109582,R-MMU-75205
R-MMU-109582,R-MMU-75892


In [10]:
reactome_pathways %>%
    select(reactome_pathway_id) %>%
    distinct()

reactome_pathway_id
<chr>
R-MMU-109581
R-MMU-109582
R-MMU-109606
R-MMU-109703
R-MMU-109704
R-MMU-110313
R-MMU-110373
R-MMU-110381
R-MMU-111458


In [11]:
train <- expression_data %>%
    filter(group == "S0R24h" | group == "S400R24h") %>%
    mutate(group = ifelse(group == "S0R24h", 1, 0)) %>%
    select(-sample_id, -repl)

train$group <- as.factor(train$group)

# remove the columns with all 0s
train <- train[, colSums(train != 0) > 0]

normalize <- function(x) {
    (x - min(x)) / (max(x) - min(x))
}

#normalize the columns
train <- train %>%
    mutate_if(negate(is.factor), normalize)

# Data export
## Training data
Store the training data in a new file

In [14]:
train %>%
    write.csv("data/train.csv", row.names = TRUE)

ERROR: Error in utils::write.table(., "data/train.csv", row.names = TRUE, row.id = NULL, : unused argument (row.id = NULL)



## Graph export
Extract the protein - pathway graph from uniprot_to_reactome, pathway - reaction graph from reactome_pathways and save it as a GraphML graph.

In [13]:
df_pr <- uniprot_to_reactome %>%
    select(uniprot_id, reactome_pathway_id) %>%
    distinct()
colnames(df_pr) <- c("from", "to")
df_rr <- reactome_pathways %>%
    select(reactome_pathway_id, reactome_reaction_id) %>%
    distinct()
colnames(df_rr) <- c("from", "to")

df <- df_pr %>%
    bind_rows(df_rr)

g <- graph_from_data_frame(df, directed = FALSE)
write.graph(g, file = "data/graph.graphml", format = "graphml")

## Matrix export
Export the protein - pathway matrix

In [15]:
uniprot_to_reactome %>%
    select(uniprot_id, reactome_pathway_id) %>%
    distinct() %>%
    write.csv("data/pp-contingency-matrix.csv", row.names = FALSE)

# Model fitting

## Fit a logistic regression model

In [12]:
model <- glm(group ~ ., data = train, family = binomial("logit"))
summary(model)


Call:
glm(formula = group ~ ., family = binomial("logit"), data = train)

Deviance Residuals: 
 [1]  0  0  0  0  0  0  0  0  0  0  0  0

Coefficients: (3715 not defined because of singularities)
              Estimate Std. Error z value Pr(>|z|)
(Intercept)     194.30 1569395.11       0        1
A0A067XG53     -140.50  714385.35       0        1
A0A075B5P4     -227.90 1594177.34       0        1
A0A075B5P5      100.93  810137.03       0        1
A0A075B5P6      -50.45  656685.75       0        1
A0A087WNV1     -120.63 1149297.05       0        1
A0A087WP47     -120.03  864350.73       0        1
A0A087WPF0      -68.91  465771.82       0        1
A0A087WPP8      -54.00  750106.88       0        1
A0A087WPT7      178.33 1680202.10       0        1
A0A087WPZ6      -75.40  774044.89       0        1
A0A087WQM0       24.63  459170.57       0        1
A0A087WQN7          NA         NA      NA       NA
A0A087WQS2          NA         NA      NA       NA
A0A087WR42          NA         NA      

## Fit a PLS model

In [ ]:
library(mixOmics)


In [ ]:
model <- plsda(group ~ ., data = train, ncomp = 2)
summary(model)

In [ ]:
plot(model, dimen = c(1))
plot(model, dimen = c(1, 2), ep = 2)

In [ ]:
# get the the attributes of the model
attributes(model)
model$

## Fit a lasso model

In [ ]:
install.packages("glmnet")


In [ ]:

library(glmnet)

In [ ]:
glmod <- glmnet(as.matrix(train[, -1]), train$group, family = "binomial", alpha = 1)
summary(glmod)

## Subset selection

In [ ]:
library(leaps)

In [ ]:
model <- regsubsets(group ~ ., data = train, nvmax = 2, really.big = TRUE, method = "forward")
summary(model)